# 本次研讨课内容:
   问答摘要内容讲解:
    1. beam search 讲解.
    2. seq2seq baseline 编写\训练\测试.
    3. 提交你的第一版成绩.
    4. 预测结果分析,模型提升改进点讨论

![](https://img-blog.csdnimg.cn/20191212183346344.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lUX2ZseWluZzYyNQ==,size_16,color_FFFFFF,t_70)

解码是seq2seq模型的常见问题，常用方法有贪心搜索`（Greedy Search）`集束搜索`（Beam Search）`。

Decoder根据Encoder的中间语义编码向量c和`<s>`标签得到第一个输出的概率分布`[0.1,0.1,0.3,0.4,0.1]`，选择概率最大的`0.4`，即`moi`。

根据隐向量h1h1和moi得到第二个输出的概率分布`[0.1,0.1,0.1,0.1,0.6][0.1,0.1,0.1,0.1,0.6]`，选择概率最大的`0.6`，即`suis`。

以此类推，直到遇到`<\s>`标签，得到最终的序列`moi suis étudiant`.

# 集束搜索

上面的贪心搜索只选择了概率最大的一个，而集束搜索则选择了概率最大的前k个。这个k值也叫做集束宽度（Beam Width）。

还是以上面的例子作为说明，k值等于2，则集束搜索的过程如下图：

![](https://img-blog.csdnimg.cn/20191212183550829.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lUX2ZseWluZzYyNQ==,size_16,color_FFFFFF,t_70)

得到第一个输出的概率分布`[0.1,0.1,0.3,0.4,0.1] [0.1,0.1,0.3,0.4,0.1]`，选择概率最大的前两个，`0.3`和`0.4`，即`Je`和`moi`。

然后`Je`和`moi`分别作为`Decoder`的输入，得到两个概率分布，然后再选择概率和最大的前两个序列，`0.3+0.8`和`0.4+0.6`，即`Je suis`和`moi suis`。

以此类推，最终可以得到两个序列，即`Je suis étudiant`和`moi suis étudiant`，很明显前者的概率和最大，为`2.2`，所以这个序列是最终得到的结果。

集束搜索本质上也是贪心的思想，只不过它考虑了更多的候选搜索空间，因此可以得到更多的翻译结果。

![](https://img-blog.csdn.net/20181011144011354?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3FxXzE2MjM0NjEz/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
集束搜索可以认为是维特比算法的贪心形式，在维特比所有中由于利用动态规划导致当字典较大时效率低，而集束搜索使用beam size参数来限制在每一步保留下来的可能性词的数量。集束搜索是在测试阶段为了获得更好准确性而采取的一种策略，在训练阶段无需使用。

假设字典为[a,b,c]，beam size选择2，则如下图有：

1：在生成第1个词的时候，选择概率最大的2个词，那么当前序列就是a或b

2：生成第2个词的时候，我们将当前序列a或b，分别与字典中的所有词进行组合，得到新的6个序列aa ab ac ba bb bc,然后从其中选择2个概率最高的，作为当前序列，即ab或bb

3：不断重复这个过程，直到遇到结束符为止。最终输出2个概率最高的序列。

# 加载训练好的模型

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append('/home/roger/kaikeba/03_lecture/code')

from utils.wv_loader import load_vocab
from utils.data_loader import load_dataset
from utils.config import *

import numpy as np
from utils.gpu_utils import config_gpu
config_gpu()
import tensorflow as tf
from seq2seq_tf2.seq2seq_model import Seq2Seq

Building prefix dict from the default dictionary ...
2020-02-09 19:36:04,173 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/cb/hnw705xd7277l6s881fs2jgm0000gp/T/jieba.cache
2020-02-09 19:36:04,180 : DEBUG : Loading model from cache /var/folders/cb/hnw705xd7277l6s881fs2jgm0000gp/T/jieba.cache
Loading model cost 2.062 seconds.
2020-02-09 19:36:06,239 : DEBUG : Loading model cost 2.062 seconds.
Prefix dict has been built successfully.
2020-02-09 19:36:06,241 : DEBUG : Prefix dict has been built successfully.


## 数据预处理

In [ ]:
# load_dataset()

In [4]:
vocab, reverse_vocab = load_vocab(vocab_path)
vocab_size=len(vocab)

In [95]:
batch_size=3

In [96]:
params = {}
params["vocab_size"] = vocab_size
params["embed_size"] = 500
params["enc_units"] = 512
params["attn_units"] = 512
params["dec_units"] = 512
params["batch_size"] = batch_size

params["beam_size"]=3

params['min_dec_steps']=4 # 
params['max_dec_steps']=50 # </s>

In [97]:
train_X,train_Y,test_X = load_dataset()

In [99]:
train_X[0]

array([31816,   415,   903,     0,   260,   241,     0,   415,    10,
          17,     2,    14,     1,     3,   103,    10,    17,     1,
           3,   260,   241,     0,   415,     1,     2,    14,     1,
           3,    17,   415,   475,     1,     3,    20,     1,     2,
          14,     1,     3,   260,   903,     0,   576,   134,     1,
           2,     7,    11,     1,     2,    18,   587,     6,     1,
           3,    60,   466,  1654,   203,     0,   394,   903,    63,
           4,     1,     2,   640,   903,     1,     3,     0, 25980,
           1,     2,    14,     1,     3,   431,     0,   329,   221,
          24,   903,     0,  1726,  4739,   241,    17,   117,     0,
         378,  5231,     0,    33,    58,     7,    81,   329,  2040,
           4,     1,     2,    14,     1,     2,    14, 31817, 31818,
       31818, 31818, 31818, 31818, 31818, 31818, 31818, 31818, 31818,
       31818, 31818, 31818, 31818, 31818, 31818, 31818, 31818, 31818,
       31818, 31818,

In [100]:
train_X.shape

(82873, 200)

# 载入模型

In [101]:
params

{'vocab_size': 31820,
 'embed_size': 500,
 'enc_units': 512,
 'attn_units': 512,
 'dec_units': 512,
 'batch_size': 3,
 'beam_size': 3,
 'min_dec_steps': 4,
 'max_dec_steps': 50}

In [102]:
model = Seq2Seq(params)

In [103]:
model

# 2 读取训练好的模型

In [104]:
from utils.config import checkpoint_dir,checkpoint_prefix

In [105]:
checkpoint_dir

'/Users/yanqiang/PycharmProjects/kaikeba/lecture_5_2/data/checkpoints/training_checkpoints_mask_loss_dim500_seq'

In [106]:
ckpt = tf.train.Checkpoint(Seq2Seq=model)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=5)
# ckpt.restore(ckpt_manager.latest_checkpoint)
# print("Model restored")

Model restored


# code 

![](https://img-blog.csdnimg.cn/20191212183346344.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lUX2ZseWluZzYyNQ==,size_16,color_FFFFFF,t_70)

In [12]:
# 定义一个类来存储每一步的中间结果
class Hypothesis:
    """ Class designed to hold hypothesises throughout the beamSearch decoding """
    def __init__(self, tokens, log_probs, hidden, attn_dists):
        self.tokens = tokens  # list of all the tokens from time 0 to the current time step t
        self.log_probs = log_probs  # list of the log probabilities of the tokens of the tokens
        self.hidden = hidden  # decoder hidden state after the last token decoding
        self.attn_dists = attn_dists  # attention dists of all the tokens
        self.abstract = ""

    def extend(self, token, log_prob, hidden, attn_dist):
        """Method to extend the current hypothesis by adding the next decoded token and all the informations associated with it"""
        return Hypothesis(tokens=self.tokens + [token],  # we add the decoded token
                          log_probs=self.log_probs + [log_prob],  # we add the log prob of the decoded token
                          hidden=hidden,  # we update the state
                          attn_dists=self.attn_dists + [attn_dist])
    @property
    def latest_token(self):
        return self.tokens[-1]

    @property
    def tot_log_prob(self):
        return sum(self.log_probs)

    @property
    def avg_log_prob(self):
        return self.tot_log_prob / len(self.tokens)

# 初始化一个对象列表

In [107]:
batch_size

3

In [108]:
enc_inp=test_X[:3]

In [109]:
enc_inp.shape

(3, 200)

In [16]:
enc_output, enc_hidden = model.call_encoder(enc_inp)

In [113]:
enc_output.shape

TensorShape([3, 200, 512])

In [18]:
enc_hidden.shape

TensorShape([3, 512])

In [116]:
enc_hidden[0]

<tf.Tensor: shape=(512,), dtype=float32, numpy=
array([-0.15255082,  0.06849127,  0.02576067, -0.21464548,  0.08560768,
        0.12540615, -0.01237227, -0.07497098, -0.0627302 , -0.2219395 ,
       -0.3586315 ,  0.03173978,  0.06696814,  0.03594634, -0.10768582,
        0.15288863, -0.2617389 , -0.10256704, -0.18194315,  0.17259544,
        0.26872373,  0.05511544,  0.33056027,  0.04681689,  0.03308366,
        0.02975553,  0.34884924, -0.07119557,  0.00269122, -0.15001512,
        0.10215709,  0.12251413,  0.19322753,  0.20117214, -0.20125249,
        0.01499808, -0.1921494 ,  0.25185302,  0.0756875 , -0.03805798,
       -0.06387903,  0.12379037,  0.0053623 , -0.05715118,  0.11909947,
        0.10920532,  0.11579174, -0.00699591,  0.13727206,  0.2802562 ,
       -0.11246926,  0.2700776 ,  0.13021748,  0.17622572,  0.21909243,
       -0.03193432,  0.22020918, -0.10747288, -0.06599473, -0.08924091,
        0.03989922,  0.04054447, -0.25073087, -0.20080546,  0.28185415,
       -0.005426

In [19]:
hyps = [Hypothesis(tokens=[vocab['<START>']],
                   log_probs=[0.0],
                   hidden=enc_hidden[0],
                   attn_dists=[],
                   ) for _ in range(params['batch_size'])]

In [20]:
# vocab {a,b,c,d,e,f}

In [22]:
# <START> 

In [114]:
hyps

In [24]:
results = []  # list to hold the top beam_size hypothesises 
steps = 0  # initial step 时间步

## 获取最新tokens 

In [25]:
latest_tokens = [h.latest_token for h in hyps]

In [26]:
latest_tokens

[31816, 31816, 31816]

## 隐藏层状态

In [27]:
hiddens = [h.hidden for h in hyps]

In [28]:
len(hiddens)

3

![](https://img-blog.csdn.net/20180414103300419)

## 单步运行decode

In [29]:
pad_index = vocab['<PAD>']
nuk_index = vocab['<UNK>']
start_index = vocab['<START>'] # 关注一个
stop_index = vocab['<STOP>']

In [117]:
start_index

31816

In [118]:
# 第一轮
steps=0

In [119]:
def decoder_onestep(enc_output,dec_input,dec_hidden):
    # 单个时间步 运行
    preds, dec_hidden, context_vector,attention_weights = model.call_decoder_onestep(dec_input,dec_hidden, enc_output)
    # 拿到top k个index 和 概率
    top_k_probs, top_k_ids = tf.nn.top_k(tf.squeeze(preds), k=params["beam_size"])
    # 计算log概率
    top_k_log_probs = tf.math.log(top_k_probs)
    # 返回需要保存的中间结果和概率
    return preds,dec_hidden,context_vector,attention_weights,top_k_log_probs,top_k_ids

## 单次搜索

In [120]:
# 计算第encoder的输出
enc_output, enc_hidden = model.call_encoder(enc_inp)

In [121]:
# 第一个decoder输入 开始标签
dec_input = tf.expand_dims(latest_tokens, 1)

In [122]:
# 第一个隐藏层输入
dec_hidden = enc_hidden

In [123]:
# 单步运行
preds, dec_hidden, context_vector,attention_weights, top_k_log_probs, top_k_ids = decoder_onestep(enc_output,dec_input,dec_hidden)

In [124]:
preds

<tf.Tensor: shape=(3, 31820), dtype=float32, numpy=
array([[-0.00768474, -0.0092312 , -0.00109153, ..., -0.01140522,
        -0.00739135, -0.00269845],
       [-0.00678893, -0.01206916, -0.00251285, ..., -0.01157113,
        -0.00701231, -0.00455805],
       [-0.00690551, -0.01211246, -0.00217122, ..., -0.01158144,
        -0.00649203, -0.00397852]], dtype=float32)>

In [126]:
params['beam_size']

3

In [ ]:
# batch_size=3,beam_size=3

In [127]:
len(hyps)

3

In [128]:
# 现阶段全部可能情况
all_hyps = [] # 9个
# 原有的可能情况数量
num_orig_hyps = 1 if steps == 0 else len(hyps)

# 遍历添加所有可能结果
for i in range(num_orig_hyps):# 1
    h, new_hidden, attn_dist = hyps[i], dec_hidden[i], attention_weights[i]
    # 分裂 添加 beam size 种可能性
    for j in range(params['beam_size']):# 3
        # 构造可能的情况
        new_hyp = h.extend(token = top_k_ids[i, j].numpy(),
                                       log_prob = top_k_log_probs[i, j],
                                       hidden = new_hidden,
                                       attn_dist = attn_dist)
        # 添加可能情况
        all_hyps.append(new_hyp)

In [130]:
all_hyps

## 排序

In [40]:
# 重置
hyps = []

In [41]:
# 按照概率来排序
sorted_hyps = sorted(all_hyps, key=lambda h: h.avg_log_prob, reverse=True)

In [42]:
# 筛选top前beam_size句话 top 3
for h in sorted_hyps:
    if h.latest_token == stop_index:
        # 长度符合预期,遇到句尾,添加到结果集
        if steps >= params['min_dec_steps']:
            results.append(h)
    else:
        # 未到结束 ,添加到假设集
        hyps.append(h)
    
    # 如果假设句子正好等于beam_size 或者结果集正好等于beam_size 就不在添加
    if len(hyps) == params['beam_size'] or len(results) == params['beam_size']:
        break
steps += 1

In [132]:
if len(results) == 0:
    results = hyps

In [44]:
# all_hyps[0].avg_log_prob
# all_hyps[1].avg_log_prob
# all_hyps[2].avg_log_prob

In [133]:
hyps_sorted = sorted(results, key=lambda h: h.avg_log_prob, reverse=True)

In [137]:
# [p.avg_log_prob for p in hyps_sorted]

In [138]:
best_hyp = hyps_sorted[0]

In [140]:
best_hyp.tokens

[31816, 26703]

In [141]:
best_hyp.abstract = " ".join([reverse_vocab[index] for index in best_hyp.tokens])

In [142]:
best_hyp.abstract

'<START> 拉扣'

References

[1] https://www.tensorflow.org/tutorials/seq2seq

[2] https://blog.csdn.net/guolindonggld/article/details/79938567

# beam search 方法整合

In [143]:
def beam_decode(model,batch,vocab, params):
    # 初始化mask
    start_index = vocab['<START>']
    stop_index = vocab['<STOP>']
    
    batch_size= params['batch_size']
    
    # 单步decoder
    def decoder_onestep(enc_output,dec_input,dec_hidden):
        # 单个时间步 运行
        preds, dec_hidden, context_vector,attention_weights = model.call_decoder_onestep(dec_input,dec_hidden, enc_output)
        # 拿到top k个index 和 概率
        top_k_probs, top_k_ids = tf.nn.top_k(tf.squeeze(preds), k=params["beam_size"])
        # 计算log概率
        top_k_log_probs = - tf.math.log(top_k_probs)
        # 返回需要保存的中间结果和概率
        return preds,dec_hidden,context_vector,attention_weights,top_k_log_probs,top_k_ids
    
    # 计算第encoder的输出
    enc_output, enc_hidden = model.call_encoder(batch)
    
    # 初始化batch size个 假设对象
    hyps = [Hypothesis(tokens=[start_index],
                   log_probs=[0.0],
                   hidden=enc_hidden[0],
                   attn_dists=[],
                   ) for _ in range(batch_size)]
    # 初始化结果集
    results = []  # list to hold the top beam_size hypothesises
    # 遍历步数
    steps = 0  # initial step
    
    # 第一个隐藏层输入
    dec_hidden = enc_hidden
    
    
    # 长度还不够 并且 结果还不够 继续搜索
    while steps < params['max_dec_steps'] and len(results) < params['beam_size']:
        # 获取最新待使用的token
        latest_tokens = [h.latest_token for h in hyps]
        # 获取所以隐藏层状态
        hiddens = [h.hidden for h in hyps]
        # 最新输入
        dec_input = tf.expand_dims(latest_tokens, 1)
        
        # 单步运行decoder 计算需要的值
        preds, dec_hidden, context_vector,attention_weights, top_k_log_probs, top_k_ids = decoder_onestep(enc_output,dec_input,dec_hidden)
        
        # 现阶段全部可能情况
        all_hyps = []
        # 原有的可能情况数量
        num_orig_hyps = 1 if steps == 0 else len(hyps)

        # 遍历添加所有可能结果
        for i in range(num_orig_hyps):
            h, new_hidden, attn_dist = hyps[i], dec_hidden[i], attention_weights[i]
            # 分裂 添加 beam size 种可能性
            for j in range(params['beam_size']):
                if params['batch_size']==1:
                    # 构造可能的情况
                    new_hyp = h.extend(token = top_k_ids[j].numpy(),
                                       log_prob = top_k_log_probs[j],
                                       hidden = new_hidden,
                                       attn_dist = attn_dist)
                else:
                    # 构造可能的情况
                    new_hyp = h.extend(token = top_k_ids[i, j].numpy(),
                                       log_prob = top_k_log_probs[i, j],
                                       hidden = new_hidden,
                                       attn_dist = attn_dist)
                # 添加可能情况
                all_hyps.append(new_hyp)
        
        # 重置
        hyps = []
        # 按照概率来排序
        sorted_hyps = sorted(all_hyps, key=lambda h: h.avg_log_prob, reverse=True)
        
        # 筛选top前beam_size句话
        for h in sorted_hyps:
            if h.latest_token == stop_index:
                # 长度符合预期,遇到句尾,添加到结果集
                if steps >= params['min_dec_steps']:
                    results.append(h)
            else:
                # 未到结束 ,添加到假设集
                hyps.append(h)

            # 如果假设句子正好等于beam_size 或者结果集正好等于beam_size 就不在添加
            if len(hyps) == params['beam_size'] or len(results) == params['beam_size']:
                break

        steps += 1
        
    if len(results) == 0:
        results = hyps
    
    hyps_sorted = sorted(results, key=lambda h: h.avg_log_prob, reverse=True)
    print(hyps_sorted)
    best_hyp = hyps_sorted[0]
    best_hyp.abstract = " ".join([reverse_vocab[index] for index in best_hyp.tokens])
    return best_hyp

In [144]:
# 读取模型
model = Seq2Seq(params)

In [145]:
# 构造数据
test_batch=test_X[:3] # beam_size=3

In [146]:
test_X[:3].shape

(3, 200)

In [147]:
test_batch.shape

(3, 200)

In [148]:
# 获得最好的语句
best_hyp=beam_decode(model,test_batch,vocab, params)

[<__main__.Hypothesis object at 0x16339a250>, <__main__.Hypothesis object at 0x14636c090>, <__main__.Hypothesis object at 0x12727c0d0>]


In [149]:
best_hyp.abstract

'<START> 说真的 会过 新门 学习机 在意 佛 那不修 提个 跟个 科鲁滋 隔行如隔山 到头 超个 大多 10.9 顿顿 科鲁滋 隔行如隔山 提个 驾龄 比亚迪G3 红框 镶 提个 跟个 比亚迪G3 奇怪的是 完怠速 科鲁滋 隔行如隔山 提个 驾龄 比亚迪G3 红框 镶 提个 跟个 比亚迪G3 奇怪的是 完怠速 科鲁滋 隔行如隔山 提个 驾龄 比亚迪G3 红框 镶 提个 跟个 比亚迪G3'

In [150]:
def test():
    def test_sub():
        print(1)

In [151]:
test_sub()

NameError: name 'test_sub' is not defined

In [152]:
def test():
    def test_sub():
        print(1)
    test_sub()
test()

1


In [153]:
def test_sub():
        print(1)
def test():
    test_sub()
test_sub()

1
